In [1]:
import numpy as np
import matplotlib.pyplot as plt

## 一、逻辑回归

In [2]:
class LR:
    def __init__(self, alpha = 0.01, iteration = 1000):
        self.alpha = alpha
        self.iteration = iteration
        self.w = None
        
    def fit(self, X_data, y_data):
        self.__gradAscent(X_data, y_data)
        
    def score(self, X_test, y_test):
        res_score = 0
        for x, y in zip(X_test, y_test):
            x = np.append(x, 1)
            pt = self.__sign(self.w @ x.T)
            res = 1 if pt > 1-pt else 0
            if res == y:
                res_score += 1

        print('right rate: %.3f' % (res_score / y_test.shape[0]))

    # 
    def __gradAscent(self, X_data, y_data):# 损失函数采用对数损失函数，其数学形式与似然函数一致
        b = np.ones((X_data.shape[0], 1))   
        X = np.hstack((X_data, b))
        w = np.ones(X.shape[1])
           
        i = 0
        while i <= self.iteration:
            err = y_data - self.__sign(w @ X.T)
            w += self.alpha * err @ X
            i += 1
        self.w = w
            
        
        
    def __sign(self, z):
        return 1.0 / (1 + np.exp(-z))

In [3]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

digits = load_digits(n_class=2)
X_data = digits['data']
y_data = digits['target']


X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=42)
lr = LR()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

right rate: 1.000


## 二、SoftMax多分类

In [10]:
class SoftMax:
    def __init__(self, alpha=0.1, lenda=1e-4, iteration=1000):
        self.alpha = alpha
        self.lenda = lenda
        self.iteration = iteration
        
        self.w = None
        self.K = None  #类的数量
        self.D = None  #数据的维度
        self.N = None  #数据总量
        
    def fit(self, X_data, y_data):
        X_data = self.__init_param(X_data, y_data)
        self.__gradDecent(X_data, y_data)
        
        
    def score(self, X_test, y_test):
        res_scroe = 0
        y_predict = self.__predict(X_test)
       
        for y, y_pred in zip(y_test, y_predict):
            res_scroe += 1 if y == y_pred else 0
        
        print('right rate: %.3f' % (res_scroe / y_test.shape[0]))

    
    def __predict(self, X):
        b = np.ones((X.shape[0], 1))
        X = np.hstack((X, b))  # 附加偏置项
        prob = np.exp(X @ self.w.T)
        return np.argmax(prob, axis=1)
    
        
    def __init_param(self, X_data, y_data):
        # 初始化，暂定输入数据全部为数值形式
        b = np.ones((X_data.shape[0], 1))
        X_data = np.hstack((X_data, b))  # 附加偏置项
        self.K = len(np.unique(y_data))
        self.D = X_data.shape[1]
        self.N = X_data.shape[0]
        self.w = np.ones((self.K, self.D))  # k*d, 针对每个类，都有一组权值参数w
        return X_data
    
     # 梯度下降训练
    def __gradDecent(self, X_data, y_data):
        step = 0
        while step < self.iteration:
            step += 1
            prob = np.exp(X_data @ self.w.T)  # n*k, 行向量存储该样本属于每个类的概率
            nf = np.transpose([prob.sum(axis=1)])  # n*1
            nf = np.repeat(nf, self.K, axis=1)  # n*k
            prob = -prob / nf  # 归一化， 此处条件符号仅方便后续计算梯度
            for i in range(self.N):
                prob[i, int(y_data[i])] += 1
            grad = -1.0 / self.N * prob.T @ X_data + self.lenda * self.w  # 梯度， 第二项为衰减项
            self.w -= self.alpha * grad

In [11]:
digits = load_digits()
X_data = digits['data']
y_data = digits['target']

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=42)
clf = SoftMax()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

right rate: 0.965
